In [8]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [22]:
data = pd.read_csv("/content/income.csv")
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
48838,64,NaN,321403,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States,<=50K.
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [24]:
print(data.isnull().sum())

age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
income              0
dtype: int64


In [25]:
data = data.dropna()
print(data.isnull().sum())

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64


In [27]:
print(data['income'].unique())

['<=50K' '>50K' '<=50K.' '>50K.']


In [28]:
# Hapus titik di akhir nilai income
data['income'] = data['income'].str.replace(r'\.$', '', regex=True)

# Konversi income ke 0 dan 1
data['income'] = data['income'].replace({"<=50K": 0, ">50K": 1})

# Cek hasilnya
print(data['income'].unique())  # Harus hanya [0, 1]

[0 1]


<ipython-input-28-bf9f1373bdd6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['income'] = data['income'].str.replace(r'\.$', '', regex=True)
<ipython-input-28-bf9f1373bdd6>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['income'] = data['income'].replace({"<=50K": 0, ">50K": 1})
<ipython-input-28-bf9f1373bdd6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [29]:
# Encoding categorical variables
categorical_cols = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
data = pd.get_dummies(data, columns=categorical_cols)

In [30]:
X = data.drop("income", axis=1)
y = data["income"]

# Normalisasi Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split Data Train-Test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [31]:
# Convert to Tensor for PyTorch
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Model Deep Learning - PyTorch
class PyTorchModel(nn.Module):
    def __init__(self, input_dim):
        super(PyTorchModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# Inisialisasi Model PyTorch
model_torch = PyTorchModel(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model_torch.parameters(), lr=0.001)

# Training PyTorch Model
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model_torch(X_train_torch)
    loss = criterion(y_pred, y_train_torch.float())  # Perbaikan disini
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Evaluasi PyTorch
y_pred_torch = (model_torch(X_test_torch).detach().numpy() > 0.5).astype(int)  # Perbaikan disini
accuracy_torch = accuracy_score(y_test, y_pred_torch)
precision_torch = precision_score(y_test, y_pred_torch)
recall_torch = recall_score(y_test, y_pred_torch)
f1_torch = f1_score(y_test, y_pred_torch)
auc_torch = roc_auc_score(y_test, y_pred_torch)

# Model Deep Learning - TensorFlow
model_tf = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

# Kompilasi Model
model_tf.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Training TensorFlow Model
model_tf.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Evaluasi TensorFlow
y_pred_tf = (model_tf.predict(X_test).ravel() > 0.5).astype(int)  # Perbaikan disini
accuracy_tf = accuracy_score(y_test, y_pred_tf)
precision_tf = precision_score(y_test, y_pred_tf)
recall_tf = recall_score(y_test, y_pred_tf)
f1_tf = f1_score(y_test, y_pred_tf)
auc_tf = roc_auc_score(y_test, y_pred_tf)

Epoch 0, Loss: 0.6382770538330078
Epoch 10, Loss: 0.5686026215553284
Epoch 20, Loss: 0.49232032895088196
Epoch 30, Loss: 0.4252406358718872
Epoch 40, Loss: 0.3858136832714081
Epoch 50, Loss: 0.3644102215766907
Epoch 60, Loss: 0.3492058217525482
Epoch 70, Loss: 0.33869829773902893
Epoch 80, Loss: 0.3325357735157013
Epoch 90, Loss: 0.3282247483730316


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8208 - loss: 0.3777
Epoch 2/100
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8526 - loss: 0.3139
Epoch 3/100
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8552 - loss: 0.3089
Epoch 4/100
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8577 - loss: 0.3055
Epoch 5/100
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8615 - loss: 0.2961
Epoch 6/100
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8624 - loss: 0.2952
Epoch 7/100
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8659 - loss: 0.2896
Epoch 8/100
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8628 - loss: 0.2921
Epoch 9/100
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8629 - loss: 0.2920
Epoch 10/100
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8701 - loss: 0.2784
Epoch 11/100
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8697 - loss: 0.2787
Epoch 12/100
1191/1

In [34]:
# Hasil Evaluasi
print("\nEvaluasi Model PyTorch")
print(f"Akurasi: {accuracy_torch:.4f}")
print(f"Presisi: {precision_torch:.4f}")
print(f"Recall: {recall_torch:.4f}")
print(f"F1-score: {f1_torch:.4f}")
print(f"AUC-ROC: {auc_torch:.4f}")

print("\nEvaluasi Model TensorFlow")
print(f"Akurasi: {accuracy_tf:.4f}")
print(f"Presisi: {precision_tf:.4f}")
print(f"Recall: {recall_tf:.4f}")
print(f"F1-score: {f1_tf:.4f}")
print(f"AUC-ROC: {auc_tf:.4f}")

# Bandingkan Model
best_model = "PyTorch" if f1_torch > f1_tf else "TensorFlow"
print(f"\nModel terbaik berdasarkan F1-score: {best_model}")


Evaluasi Model PyTorch
Akurasi: 0.8388
Presisi: 0.7160
Recall: 0.5771
F1-score: 0.6391
AUC-ROC: 0.7509

Evaluasi Model TensorFlow
Akurasi: 0.8279
Presisi: 0.6689
Recall: 0.6021
F1-score: 0.6337
AUC-ROC: 0.7521

Model terbaik berdasarkan F1-score: PyTorch
